In [30]:
def find_in_db(text: str) -> int:
    return -1

def set_node_id(node: dict, last_id: int, arr: []) -> int:
    if 'question' in node:
        last_id = last_id + 1
        node['question']['id'] = last_id
        arr.append((last_id, node['question']['text']))
        last_id = set_node_id(node['question']['yes'], last_id, arr)
        last_id = set_node_id(node['question']['no'], last_id, arr)
        return last_id
    else:
        text = node['diagnosis']['text']
        diagnosis_id = find_in_db(text)
        if diagnosis_id == -1:
            node['diagnosis']['id'] = last_id + 1
            arr.append((last_id + 1, node['diagnosis']['text']))
            return last_id + 1
        else:
            node['diagnosis']['id'] = diagnosis_id
            arr.append((last_id, node['diagnosis']['text']))
            return last_id

def get_last_id() -> int:
    return 142


def create_insert_values(node: dict, arr: []):
    if 'question' in node:
        yes_type = list(node['question']['yes'].keys())[0]
        no_type =  list(node['question']['no'].keys())[0]
        arr.append(
            (node['question']['id'],
             node['question']['text'],
             node['question']['yes'][yes_type]['id'],
             node['question']['no'][no_type]['id'])
        )
        create_insert_values(node['question']['yes'], arr)
        create_insert_values(node['question']['no'], arr)
    else:
        arr.append(
            (node['diagnosis']['id'],
             node['diagnosis']['text'],
             'null',
             'null',
             node['diagnosis']['doc'],
             node['diagnosis']['description']
             )
        )

def create_inserts(file: dict) -> str:
    result = []
    to_remove = {}
    page_id = file['page']['id']
    page_text = file['page']['page_name']
    create_insert_values(file['page']['tree'], result)

    # исключим повторения
    counter = {}
    for elem in [text[1] for text in result]:
        counter[elem] = counter.get(elem, 0) + 1
    doubles = {element: count for element, count in counter.items() if count > 1}
    for repeat in doubles:
        arr = [elem for elem in result if elem[1] == repeat]
        main_elem = arr[0]
        to_remove[main_elem[0]] = []
        for i in range(1, len(arr)):
            to_remove[main_elem[0]].append(arr[i][0])
            result.remove(arr[i])

    # создаем строку
    result_str = f"INSERT INTO main_tree VALUES({page_id}, '{page_text}', {result[0][0]}, {result[0][0]});\n"
    for elem in result:
        if len(elem) == 4:
            result_str += f"INSERT INTO main_tree VALUES({elem[0]}, '{elem[1]}', {elem[2]}, {elem[3]});\n"
        else:
            result_str += f"INSERT INTO main_tree VALUES({elem[0]}, '{elem[1]}', {elem[2]}, {elem[3]}, '{elem[4]}', '{elem[5]}');\n"

    # меняем индексы родителей в строке
    for elem in to_remove:
        for old_elem in to_remove[elem]:
            result_str = result_str.replace(str(old_elem), str(elem))
    return result_str


def parse(file: dict):
    last_id = get_last_id()
    file['page']['id'] = last_id + 1
    result = []
    set_node_id(file['page']['tree'], last_id + 1, result)
    result1 = create_inserts(file)
    return result1



In [32]:
import json


with open("1.json", "r", encoding='utf8') as read_file:
    b: {} = json.load(read_file)

a = parse(b)

with open("result.txt", "w", encoding='utf8') as file:
    file.write(a)